In [6]:
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib widget
%matplotlib qt

In [8]:
dirName = '/home/valentin/crazyflie/Recordings/tube/'
fileNameList = ['sd_33_cf2414.csv']

data =  pd.read_csv(dirName + fileNameList[0])



In [10]:
tim = data['tick']/1000
# tim = tim - tim[0]
# ind = np.where(tim > 150)
# print(str(tim))
plt.figure()
# plt.subplot(2,1,1)
# plt.plot(tim, data['dvio_data.velStdDevx'],linewidth=3)
# plt.plot(tim, data['dvio_data.velStdDevy'],linewidth=3)
# plt.plot(tim, data['dvio_data.velStdDevz'],linewidth=3)
# plt.plot(tim, data['state_machine.ExtKalman'],linewidth=3)
# plt.plot(tim, data['kalman.SensorConfig'],linewidth=3)
# plt.plot(tim, data['kalman_states.useVel'],linewidth=3)

# plt.plot(tim, data['stateEstimate.vx'],linewidth=3)
# plt.plot(tim, data['kalman.statePX'],linewidth=3)
# plt.plot(tim, data['range.valid'],linewidth=3)
plt.plot(tim, data['range.zref'],linewidth=3)
# plt.plot(tim, data['range.ztrans'],linewidth=3)
# plt.plot(tim, data['range.zmeas'],linewidth=3)
# plt.plot(tim, data['range.vzTof'],linewidth=3)
# plt.plot(tim, data['range.vzKal'],linewidth=3)
# plt.plot(tim, data['stateEstimate.z'],linewidth=3)
# plt.plot(tim, data['posCtl.targetZ']*100,linewidth=3)

plt.legend(['zref','ztrans','zmeas','vzTof','vzKal','stateEstimate.z'])
# plt.plot(tim, data['stateEstimate.z'],linewidth=3)
# plt.plot(tim, data['dvio_data.stdFacZ'],linewidth=3)


# plt.plot(tim[1:], np.abs(np.diff(data['kalman_pred.tof_innov'])>0))
# plt.plot(tim[ind], data['kalman.stateY'][ind],linewidth=3)
# plt.plot(tim[ind], data['kalman.stateY_f'][ind],linewidth=3) #stateEstimate.z
# plt.plot(data['kalman.stateX'][ind], data['kalman.stateY'][ind],linewidth=3) #stateEstimate.z
# plt.plot(data['kalman.stateX_f'][ind], data['kalman.stateY_f'][ind],linewidth=3) #stateEstimate.z
plt.xlabel('Time [s]')
plt.ylabel('Position [m]')
plt.grid(True)

# plt.subplot(2,1,2)
# plt.plot(tim, data['motor.m1'],linewidth=3)
# plt.xlabel('Time [s]')
# plt.ylabel('Motor [PWM]')
# plt.grid(True)
plt.show()

KeyError: 'range.zref'

In [5]:
import numpy as np
from scipy.signal import hann, fftconvolve
from scipy.fftpack import fft, ifft

def PTstepcalc(SP, GY, lograte, Ycorrection, smoothFactor):
    """
    This function deconvolves the step response function using
    SP = set point (input), GY = filtered gyro (output)
    returns matrix/stack of estimated stepresponse functions, time [t] in ms
    """
    # "THE BEER-WARE LICENSE" (Revision 42):
    # <brian.white@queensu.ca> wrote this file. As long as you retain this notice you
    # can do whatever you want with this stuff. If we meet some day, and you think
    # this stuff is worth it, you can buy me a beer in return. -Brian White

    from scipy.signal import savgol_filter
    smoothVals = [1, 20, 40, 60]
    GY = savgol_filter(GY, smoothVals[smoothFactor], 3)  # 'lowess' equivalent in python

    minInput = 20
    segment_length = (lograte*2000)  # 2 sec segments
    wnd = (lograte*1000) * .5  # 500ms step response function, length will depend on lograte  
    StepRespDuration_ms = 500  # max dur of step resp in ms for plotting
    t = np.arange(0, StepRespDuration_ms + 1/lograte, 1/lograte)  # time in ms 

    fileDurSec = len(SP) / (lograte*1000)
    subsampleFactor = 1
    if fileDurSec <= 20:
        subsampleFactor = 10
    elif 20 < fileDurSec <= 60:
        subsampleFactor = 7
    elif fileDurSec > 60:
        subsampleFactor = 3

    segment_vector = np.arange(0, len(SP), round(segment_length/subsampleFactor))
    NSegs = np.max(np.where((segment_vector+segment_length) < segment_vector[-1]))
    if NSegs > 0:
        SPseg = []
        GYseg = []
        j = 0
        for i in range(NSegs):
            if np.max(np.abs(SP[segment_vector[i]:segment_vector[i]+segment_length])) >= minInput:
                j += 1
                SPseg.append(SP[segment_vector[i]:segment_vector[i]+segment_length])
                GYseg.append(GY[segment_vector[i]:segment_vector[i]+segment_length])

        padLength = 100  # 2^nextpow2(length(SPseg[i,:]))
        j = 0
        if SPseg:
            for i in range(len(SPseg)):
                a = GYseg[i] * hann(len(GYseg[i]))
                b = SPseg[i] * hann(len(SPseg[i]))
                a = fft(np.concatenate([np.zeros(padLength), a, np.zeros(padLength)]))
                b = fft(np.concatenate([np.zeros(padLength), b, np.zeros(padLength)]))
                G = a / len(a)
                H = b / len(b)
                Hcon = np.conj(H)

                imp = np.real(ifft((G * Hcon) / (H * Hcon + 0.0001)))  # impulse response function
                resptmp = np.cumsum(imp)  # integrate impulse resp function 

                steadyStateWindow = np.where((t > 200) & (t < StepRespDuration_ms))
                steadyStateResp = resptmp[steadyStateWindow]
                if Ycorrection:
                    if np.nanmean(steadyStateResp) < 1 or np.nanmean(steadyStateResp) > 1:
                        yoffset = 1 - np.nanmean(steadyStateResp)
                        resptmp = resptmp * (yoffset+1)
                    steadyStateResp = resptmp[steadyStateWindow]

                if np.min(steadyStateResp) > 0.5 and np.max(steadyStateResp) < 3:  # Quality control     
                    j += 1
                    stepresponse = resptmp[:1+wnd]
    return stepresponse, t


ModuleNotFoundError: No module named 'scipy'